# Graphene Nanoribbons (GNR) with Tight‑Binding + NEGF (ASE only)

**Audience:** Undergraduate mini‑project  
**Goal:** Build **graphene**, **armchair/zigzag nanoribbons**, compute **bandstructure**, **Hamiltonian matrices**, **DOS**, **transmission**, and a simple **bond‑current** quantity — all **from scratch** (no QuantumATK/SIESTA).

> Nearest‑neighbor pz tight‑binding (hopping `t=-2.7 eV`) built directly from ASE geometries.

---

## Notebook outline

1. Install & imports  
2. Graphene unit cell + analytic bandstructure (Γ–K–M–Γ)  
3. Build GNRs (armchair/zigzag), change width and repeats, visualize & save  
4. Build TB Hamiltonian blocks `H0` and `H1` from the geometry  
5. Ribbon bandstructure from `H(k)=H0+H1e^{ik}+H1†e^{-ik}`  
6. NEGF transmission for one ribbon device  
7. Transmission vs width  
8. Visualize the Hamiltonian matrices  
9. DOS from Green’s function  
10. Simple bond‑current matrix at a chosen energy (teaching tool)


In [ ]:
# Cell 0: Install & imports
!pip -q install ase scipy matplotlib

import numpy as np
import matplotlib.pyplot as plt

from ase.build import graphene, graphene_nanoribbon
from ase.io import write
from ase.visualize.plot import plot_atoms
from ase.neighborlist import neighbor_list

from scipy.linalg import inv, norm

import ase, scipy
print("ASE:", ase.__version__)
print("SciPy:", scipy.__version__)


## 1) Graphene unit cell + analytic tight‑binding bandstructure

We build a graphene unit cell with ASE and then plot the **analytic** nearest‑neighbor TB bandstructure.

Hamiltonian in k‑space (2×2):
\[
H(\mathbf{k})=
\begin{pmatrix}
0 & t f(\mathbf{k})\\
t f^*(\mathbf{k}) & 0
\end{pmatrix},
\quad
f(\mathbf{k})=\sum_{n=1}^3 e^{i\mathbf{k}\cdot \boldsymbol{\delta}_n}
\]


In [ ]:
# Cell 1A: Graphene unit cell (ASE)
a = 2.46  # Å lattice constant for graphene
gr = graphene(a=a, size=(1, 1, 1), vacuum=10.0)  # correct ASE signature
gr.pbc = [True, True, False]

print(gr)
fig, ax = plt.subplots(figsize=(4,4))
plot_atoms(gr, ax, radii=0.3)
ax.set_title("Graphene unit cell (ASE)")
plt.show()

write("graphene_unitcell.xyz", gr)
print("Saved: graphene_unitcell.xyz")


In [ ]:
# Cell 1B: Analytic graphene TB bandstructure (Γ–K–M–Γ)

t = -2.7  # eV nearest-neighbor hopping

# Nearest-neighbor vectors (A -> B) in 2D
d1 = np.array([0.0, a/np.sqrt(3)])
d2 = np.array([ a/2, -a/(2*np.sqrt(3))])
d3 = np.array([-a/2, -a/(2*np.sqrt(3))])

def Hk_graphene(kx, ky):
    k = np.array([kx, ky])
    f = np.exp(1j*np.dot(k, d1)) + np.exp(1j*np.dot(k, d2)) + np.exp(1j*np.dot(k, d3))
    return np.array([[0, t*f],
                     [t*np.conjugate(f), 0]], dtype=complex)

# Reciprocal lattice vectors (standard)
b1 = (2*np.pi/a)*np.array([1, -1/np.sqrt(3)])
b2 = (2*np.pi/a)*np.array([0,  2/np.sqrt(3)])

G = np.array([0.0, 0.0])
K = (b1 + 2*b2)/3
M = (b1 + b2)/2

path = [G, K, M, G]
labels = [r'$\Gamma$', r'$K$', r'$M$', r'$\Gamma$']
nseg = 120

kpts, kdist = [], [0.0]
xticks = [0.0]
for i in range(len(path)-1):
    k0, k1 = path[i], path[i+1]
    for j in range(nseg):
        s = j/(nseg-1)
        k = (1-s)*k0 + s*k1
        if kpts:
            kdist.append(kdist[-1] + np.linalg.norm(k - kpts[-1]))
        kpts.append(k)
    xticks.append(kdist[-1])

E = []
for k in kpts:
    w = np.linalg.eigvalsh(Hk_graphene(k[0], k[1]))
    E.append(np.sort(np.real(w)))
E = np.array(E)

plt.figure(figsize=(6,4))
plt.plot(kdist, E[:,0], lw=2)
plt.plot(kdist, E[:,1], lw=2)
for x in xticks:
    plt.axvline(x, color='k', lw=0.5)
plt.xticks(xticks, labels)
plt.ylabel("Energy (eV)")
plt.title("Graphene TB bandstructure (nearest-neighbor)")
plt.grid(alpha=0.3)
plt.show()


## 2) Build nanoribbons (armchair & zigzag), change width and repeat, visualize & save

ASE provides a constructor:
- `type="armchair"` or `type="zigzag"`
- width parameter: `N`
- repeat along transport direction: `m`


In [ ]:
# Cell 2: Build and visualize nanoribbons

def make_ribbon(kind="armchair", N=10, m=1, vacuum=10.0):
    rib = graphene_nanoribbon(n=N, m=m, type=kind, vacuum=vacuum, saturated=False)
    return rib

ribA = make_ribbon("armchair", N=10, m=1)
ribZ = make_ribbon("zigzag",   N=10, m=1)

for name, rib in [("Armchair", ribA), ("Zigzag", ribZ)]:
    fig, ax = plt.subplots(figsize=(10,3))
    plot_atoms(rib, ax, radii=0.25, rotation=('0x,90y,0z'))
    ax.set_title(f"{name} GNR (N=10, m=1)")
    plt.show()

write("GNR_armchair_N10_m1.xyz", ribA)
write("GNR_zigzag_N10_m1.xyz", ribZ)
print("Saved: GNR_armchair_N10_m1.xyz, GNR_zigzag_N10_m1.xyz")


## 3) Build tight‑binding Hamiltonian blocks `H0` and `H1`

We construct:
- `H0`: couplings **inside** one unit cell  
- `H1`: coupling from cell `0` to cell `+1` (along x transport)

Nearest neighbors are found using ASE `neighbor_list`.


In [ ]:
# Cell 3: TB blocks from ASE geometry (neighbor-list based)

def tb_blocks_from_ase_1D(atoms, t=-2.7, cutoff=1.8, transport_axis=0):
    # Build nearest-neighbor TB blocks H0 (within cell) and H1 (cell -> cell+1)
    atoms = atoms.copy()
    pbc = atoms.get_pbc()
    if not pbc[transport_axis]:
        raise ValueError("System must be periodic along transport_axis (pbc True).")

    n = len(atoms)
    H0 = np.zeros((n,n), dtype=complex)
    H1 = np.zeros((n,n), dtype=complex)
    np.fill_diagonal(H0, 0.0)

    i_idx, j_idx, S = neighbor_list("ijS", atoms, cutoff)

    for i, j, shift in zip(i_idx, j_idx, S):
        if i == j:
            continue
        s = int(shift[transport_axis])
        if s == 0:
            H0[i, j] = t
        elif s == +1:
            H1[i, j] = t
        # s == -1 is implied by H1^\dagger

    return H0, H1

rib = make_ribbon("armchair", N=10, m=1)
H0, H1 = tb_blocks_from_ase_1D(rib, t=-2.7, cutoff=1.8, transport_axis=0)

print("Atoms per unit cell:", len(rib))
print("Nonzeros H0:", np.count_nonzero(np.abs(H0) > 1e-12))
print("Nonzeros H1:", np.count_nonzero(np.abs(H1) > 1e-12))


## 4) Ribbon bandstructure from `H(k)`

\[
H(k) = H_0 + H_1 e^{ik} + H_1^\dagger e^{-ik}
\]


In [ ]:
# Cell 4: Ribbon bandstructure

def ribbon_bands_from_blocks(H0, H1, nk=250):
    ks = np.linspace(-np.pi, np.pi, nk)
    bands = []
    for k in ks:
        Hk = H0 + H1*np.exp(1j*k) + H1.conj().T*np.exp(-1j*k)
        w = np.linalg.eigvalsh(Hk)
        bands.append(np.sort(np.real(w)))
    return ks, np.array(bands)

ks, bands = ribbon_bands_from_blocks(H0, H1, nk=250)

plt.figure(figsize=(7,4))
for b in range(bands.shape[1]):
    plt.plot(ks, bands[:,b], lw=0.8)
plt.xlabel("k (dimensionless)")
plt.ylabel("Energy (eV)")
plt.title("GNR bandstructure (from H0/H1)")
plt.grid(alpha=0.3)
plt.show()


## 5) NEGF transmission for one ribbon device

We connect a device of `Nd` cells to identical semi‑infinite leads built from the same `H0, H1`.

Lead self‑energy is computed by a stable fixed‑point iteration with linear mixing:
\[
\Sigma \leftarrow (1-\alpha)\Sigma + \alpha \, H_1^\dagger (E-H_0-\Sigma)^{-1} H_1
\]


In [ ]:
# Cell 5: NEGF transmission

def lead_self_energy(E, H0, H1, eta=1e-5, maxiter=800, tol=1e-10, mix=0.3):
    n = H0.shape[0]
    zI = (E + 1j*eta) * np.eye(n)
    Sigma = np.zeros((n,n), dtype=complex)

    for _ in range(maxiter):
        Gs = inv(zI - H0 - Sigma)
        Sigma_fp = H1.conj().T @ Gs @ H1
        Sigma_new = (1-mix)*Sigma + mix*Sigma_fp
        if norm(Sigma_new - Sigma) / (norm(Sigma_new) + 1e-15) < tol:
            Sigma = Sigma_new
            break
        Sigma = Sigma_new
    return Sigma

def build_device_H(H0, H1, Nd):
    n = H0.shape[0]
    dim = Nd*n
    Hd = np.zeros((dim, dim), dtype=complex)
    for c in range(Nd):
        Hd[c*n:(c+1)*n, c*n:(c+1)*n] = H0
        if c < Nd-1:
            Hd[c*n:(c+1)*n, (c+1)*n:(c+2)*n] = H1
            Hd[(c+1)*n:(c+2)*n, c*n:(c+1)*n] = H1.conj().T
    return Hd

def transmission_negf(E, H0, H1, Nd=6, eta=1e-5):
    n = H0.shape[0]
    Hd = build_device_H(H0, H1, Nd)
    dim = Hd.shape[0]

    Sigma0 = lead_self_energy(E, H0, H1, eta=eta)

    SigmaL = np.zeros((dim, dim), dtype=complex)
    SigmaR = np.zeros((dim, dim), dtype=complex)
    SigmaL[:n, :n] = Sigma0
    SigmaR[-n:, -n:] = Sigma0

    zI = (E + 1j*eta) * np.eye(dim)
    G = inv(zI - Hd - SigmaL - SigmaR)

    GammaL = 1j*(SigmaL - SigmaL.conj().T)
    GammaR = 1j*(SigmaR - SigmaR.conj().T)

    T = np.real(np.trace(GammaL @ G @ GammaR @ G.conj().T))
    return T, G, Hd, SigmaL, SigmaR

# Transmission spectrum
Egrid = np.linspace(-2.0, 2.0, 401)
Tvals = []
for E in Egrid:
    T, *_ = transmission_negf(E, H0, H1, Nd=6, eta=1e-5)
    Tvals.append(T)
Tvals = np.array(Tvals)

plt.figure(figsize=(7,4))
plt.plot(Egrid, Tvals, lw=2)
plt.xlabel("Energy (eV)")
plt.ylabel("Transmission T(E)")
plt.title("NEGF transmission of a GNR device")
plt.grid(alpha=0.3)
plt.show()


## 6) Transmission vs width

Compute `T(EF)` for different widths `N`.


In [ ]:
# Cell 6: Transmission at EF vs width

def T_at_EF_vs_width(kind="armchair", widths=(6,8,10,12,14,16), EF=0.0, Nd=6):
    Tw = []
    for N in widths:
        rib = make_ribbon(kind=kind, N=N, m=1)
        H0w, H1w = tb_blocks_from_ase_1D(rib, t=-2.7, cutoff=1.8, transport_axis=0)
        T, *_ = transmission_negf(EF, H0w, H1w, Nd=Nd, eta=1e-5)
        Tw.append(T)
        print(f"{kind:8s} N={N:2d} atoms/cell={len(rib):3d}  T(EF)={T:.3f}")
    return np.array(Tw)

widths = [6,8,10,12,14,16]
Tw = T_at_EF_vs_width("armchair", widths, EF=0.0, Nd=6)

plt.figure(figsize=(6,4))
plt.plot(widths, Tw, marker='o', lw=2)
plt.xlabel("Ribbon width parameter N")
plt.ylabel("T(EF)")
plt.title("Transmission at EF vs Width")
plt.grid(alpha=0.3)
plt.show()


## 7) Hamiltonian matrices

Plot the magnitude of `H0` and `H1`.


In [ ]:
# Cell 7: Visualize |H0| and |H1|

plt.figure(figsize=(5,4))
plt.imshow(np.abs(H0), interpolation="nearest")
plt.colorbar(label="|H0| (eV)")
plt.title("Intra-cell Hamiltonian |H0|")
plt.show()

plt.figure(figsize=(5,4))
plt.imshow(np.abs(H1), interpolation="nearest")
plt.colorbar(label="|H1| (eV)")
plt.title("Inter-cell coupling |H1| (cell -> cell+1)")
plt.show()


## 8) DOS from Green’s function

\[
DOS(E)= -\frac{1}{\pi}\,Im(Tr(G^r(E)))
\]


In [ ]:
# Cell 8: DOS

def dos_from_G(G):
    return -(1/np.pi) * np.imag(np.trace(G))

DOS = []
for E in Egrid:
    _, G, *_ = transmission_negf(E, H0, H1, Nd=6, eta=1e-5)
    DOS.append(dos_from_G(G))
DOS = np.array(DOS)

plt.figure(figsize=(7,4))
plt.plot(Egrid, DOS, lw=2)
plt.xlabel("Energy (eV)")
plt.ylabel("DOS (arb.)")
plt.title("Device DOS from retarded Green's function")
plt.grid(alpha=0.3)
plt.show()


## 9) Simple bond‑current matrix at a chosen energy (toy)

Construct:
\[
G^< = i\, G^r (\Gamma_L f_L + \Gamma_R f_R) (G^r)^\dagger
\]
and a bond‑current indicator:
\[
J_{ij}(E) \propto 2\,Im(H_{ij} G^<_{ji})
\]


In [ ]:
# Cell 9: Bond-current matrix (toy, energy-resolved)

def fermi(E, mu, kT):
    return 1.0/(1.0 + np.exp((E-mu)/(kT+1e-15)))

def bond_current_matrix(E, H0, H1, Nd=6, muL=0.05, muR=-0.05, kT=0.002, eta=1e-5):
    T, G, Hd, SigmaL, SigmaR = transmission_negf(E, H0, H1, Nd=Nd, eta=eta)

    GammaL = 1j*(SigmaL - SigmaL.conj().T)
    GammaR = 1j*(SigmaR - SigmaR.conj().T)

    fL = fermi(E, muL, kT)
    fR = fermi(E, muR, kT)

    Gless = 1j * (G @ (GammaL*fL + GammaR*fR) @ G.conj().T)

    J = np.zeros_like(Hd, dtype=float)
    dim = Hd.shape[0]
    for i in range(dim):
        for j in range(dim):
            if i != j and np.abs(Hd[i,j]) > 1e-12:
                J[i,j] = 2*np.imag(Hd[i,j] * Gless[j,i])
    return J, T

E0 = 0.0
J, T0 = bond_current_matrix(E0, H0, H1, Nd=6)
print("T(E=0) =", T0)

plt.figure(figsize=(6,5))
plt.imshow(J, interpolation="nearest")
plt.colorbar(label="J_ij(E=0) (arb.)")
plt.title("Bond-current matrix at E=0 (device)")
plt.show()


## Quick exercises

1. **Switch to zigzag**: set `rib = make_ribbon("zigzag", N=10, m=1)` and rerun Cells 3–9.  
2. **Change width**: modify `widths` list and see the trend.  
3. **Change device length**: vary `Nd` in transmission calculation.  
4. **Compare bandstructure vs transmission**: where do you see channels opening?

---

### Saving geometries
You can save any ASE structure with:
```python
write("my_structure.xyz", atoms)
```
